__LLM Agent using LangChain__ 

This notebook, illustrate the creation of a simple AI Agent using an OpenAI model. We will use LangChain to allow the agent access to external data and tools. 

__LangChain__ 
- LangChain is a framework designed to connect LLM's with external data sources as well as computational tools
- The core idea is chains. By chaining a series of actions together, an AI Agent can accomplish more complex tasks

First, we will load the LLM. Next we will define the tools that agent can use. Next we will create the AI Agent and give it a task. 

<div style="color:#0b3d91">

Este cuaderno ilustra la creación de un Agente de IA simple usando un modelo de OpenAI. Usaremos LangChain para permitir que el agente acceda a datos y herramientas externas.

__LangChain__
- LangChain es un framework diseñado para conectar LLMs con fuentes de datos externas y herramientas computacionales
- La idea central son las 'chains' (cadenas). Al encadenar una serie de acciones, un Agente de IA puede realizar tareas más complejas

Primero cargaremos el LLM. Luego definiremos las herramientas que el agente puede usar. Después crearemos el Agente de IA y le asignaremos una tarea.
</div>

In [7]:
# Import necessary libraries
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI
from langchain import hub

The following cell loads the LLM we will use. We will use the gpt-4o model with a temperature of zero. 

Please note, rerunning this notebook will note work unless you set up your own connection to OpenAI models.

<div style="color:#0b3d91">

La siguiente celda carga el LLM que usaremos. Usaremos el modelo gpt-4o con temperatura cero.

Tenga en cuenta que volver a ejecutar este cuaderno no funcionará a menos que configure su propia conexión a los modelos de OpenAI.
</div>

In [5]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print(f"Using LLM: {llm.model_name}")

Using LLM: gpt-4o


The following cells defines tools that the agent will utilize. For this example, the agent will be given tools to search Wikipedia. The __WikipediaAPIWrapper__ is a LangChain community tool that sets up a connection to Wikipedia. The wrapper will return the most relevant result and use up to 1000 characters to limit token usage. The __WikipediaQueryRun__ converts the wrapper into a tool that the agent can call. In this example we only have one tool, but agents can have many tools stored in a list as shown below. 

<div style="color:#0b3d91">

Las siguientes celdas definen las herramientas que el agente utilizará. Para este ejemplo, al agente se le darán herramientas para buscar en Wikipedia. El __WikipediaAPIWrapper__ es una herramienta de la comunidad de LangChain que configura una conexión a Wikipedia. El wrapper devolverá el resultado más relevante y usará hasta 1000 caracteres para limitar el uso de tokens. El __WikipediaQueryRun__ convierte el wrapper en una herramienta que el agente puede invocar. En este ejemplo solo tenemos una herramienta, pero los agentes pueden tener muchas herramientas almacenadas en una lista como se muestra a continuación.
</div>

In [8]:
# First, set up the Wikipedia API wrapper
wikipedia_api_wrapper = WikipediaAPIWrapper(top_results=1, doc_content_chars_max=1000)

# Then, create the tool itself
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

# We'll put all our tools in a list
tools = [wikipedia_tool]

print(f"Agent has {len(tools)} tool(s) available: {[tool.name for tool in tools]}")

Agent has 1 tool(s) available: ['wikipedia']


Now that the agent has a tool, we will prompt the agent. In order to do so, the agent needs instructions. LangChain provides pre-built prompts that help guide an agent on what tools to use and how to respond. The cell below uses a standard one from LangChain Hub. 

<div style="color:#0b3d91">

Ahora que el agente tiene una herramienta, lo solicitaremos mediante un prompt. Para ello, el agente necesita instrucciones. LangChain proporciona prompts preconstruidos que ayudan a guiar al agente sobre qué herramientas usar y cómo responder. La celda siguiente usa uno estándar del LangChain Hub.
</div>

In [9]:
# Get the prompt for our agent from LangChain Hub
# This prompt tells the LLM how to reason and use tools
prompt = hub.pull("hwchase17/react")

print("Agent prompt loaded. It guides the LLM on how to reason and use tools.")
# You can uncomment the line below to see the prompt template:
print(prompt.template)

Agent prompt loaded. It guides the LLM on how to reason and use tools.
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


Now we will create the agent by passing the llm, tools and the prompt to be used. 

<div style="color:#0b3d91">

Ahora crearemos el agente pasando el llm, las herramientas y el prompt a usar.
</div>

In [10]:
agent = create_react_agent(llm, tools, prompt)

print("Agent created!")

Agent created!


Now we will run the agent. The question we will ask it is: 

- __What is Lionel Messi's middle name? And what city was he born in?__


The components below have certain responsibilities: 

- __Agent Executor__ : The engine that executes the  agent's decisions
- __agent=agent, tools=tools__: The agent logic and tools it has access to

<div style="color:#0b3d91">

Ahora ejecutaremos el agente. La pregunta que le haremos es:

- __¿Cuál es el segundo nombre de Lionel Messi? ¿Y en qué ciudad nació?__

Los componentes que siguen tienen ciertas responsabilidades:

- __Agent Executor__: El motor que ejecuta las decisiones del agente
- __agent=agent, tools=tools__: La lógica del agente y las herramientas a las que tiene acceso
</div>

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

print("Running the agent with a question:")
response = agent_executor.invoke({"input": "What is Lionel Messi's middle name? And what city was he born in?"})

print("\n--- Agent's Final Answer ---")
print(response["output"])

Running the agent with a question:


> Entering new AgentExecutor chain...
Lionel Messi is a well-known figure, and I can provide information about him. However, to ensure accuracy, I will verify his middle name and birthplace.

Action: wikipedia
Action Input: Lionel MessiPage: Lionel Messi
Summary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players in history, Messi has set numerous records for individual accolades won throughout his professional footballing career, including eight Ballon d'Ors, six European Golden Shoes, and eight times being named the world's best player by FIFA. In 2025, he was named the All Time Men's World Best Player by the IFFHS. He is the most decorated player in the history of professional football having won 45 team 

As seen above, the agent call wikipedia and searches for Lionel Messi. Within the wikipedia page, it is able to find his middle name and the city in which he was born. Lastly, the agent outputs the response. 

This is a simple example, with one chain but serves as an introduction to LangChain and the concepts that make it a powerful tool. 

<div style="color:#0b3d91">

Como se ve arriba, el agente llama a Wikipedia y busca a Lionel Messi. Dentro de la página de Wikipedia, puede encontrar su segundo nombre y la ciudad en la que nació. Finalmente, el agente muestra la respuesta.

Este es un ejemplo simple, con una sola cadena, pero sirve como introducción a LangChain y a los conceptos que lo convierten en una herramienta poderosa.
</div>

__Additional Considerations__ 

In real world applications LangChain serves as a powerful tool used for different purposes. 

1. RAG (Retrieval Augmented Generation)
- LangChain helps load a company's private data into readable context for an LLM 
- LangChain helps split this data into chunks and stores them in a Vector Store, which are efficient queried by an LLM 
- The RAG system is then able to efficiently retrieve and answer questions based on context provided 

2. Querying Database
- LangChain can connect and LLM to a private database 
- Given the database structure, the LLM can write code to pull data 
- Therefore, a person can query a database in english and the LLM can correctly return the information desired 
- The impact, is less time writing basic SQL queries and empowering people to make decisions 

3. Agentic Workflows 
- By utilizing the concept of chains, an agent can complete multi-step complex tasks
- By provided tools to an LLM through LangChain, an LLM take actions based on its decisions 
- Simple workflows such as booking a hotel or creating a reservation to a restaurant are already available due to tools like LangChain. 



<div style="color:#0b3d91">

__Consideraciones Adicionales__

En aplicaciones del mundo real, LangChain actúa como una herramienta potente usada para diferentes propósitos.

1. RAG (Retrieval Augmented Generation)
- LangChain ayuda a cargar los datos privados de una empresa en un contexto legible para un LLM
- LangChain ayuda a dividir estos datos en fragmentos y los almacena en un Vector Store, que son consultados eficientemente por un LLM
- El sistema RAG puede recuperar y responder preguntas basadas en el contexto provisto

2. Consultas a Bases de Datos
- LangChain puede conectar un LLM a una base de datos privada
- Dada la estructura de la base de datos, el LLM puede escribir código para extraer datos
- Por lo tanto, una persona puede consultar una base de datos en inglés y el LLM puede devolver correctamente la información deseada
- El impacto es menos tiempo dedicado a escribir consultas SQL básicas y empoderar a las personas para tomar decisiones

3. Flujos de Trabajo Agénticos
- Al utilizar el concepto de chains, un agente puede completar tareas complejas de múltiples pasos
- Al proporcionar herramientas a un LLM a través de LangChain, un LLM puede tomar acciones basadas en sus decisiones
- Flujos simples como reservar un hotel o crear una reserva en un restaurante ya están disponibles gracias a herramientas como LangChain.
</div>